In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import HTML

In [2]:
from bqplot import Figure, Scatter, Axis, LinearScale

In [3]:
EXPLANATION = """\
<div class="app-sidebar">
<p><em>Compare different development indicators.</em><p>

<p>Select what indicators to plot in the dropdowns, and use the slider
to sub-select a fraction of years to include in the plot.</p>

<p>Data and idea copied from the <a href="https://dash.plot.ly/getting-started-part-2">
Plotly Dash documentation</a>.</p>

<p>This example demonstrates combining <a href="https://bqplot.github.io/bqplot">
bqplot</a> with Jupyter widgets.</p>
</div>
"""

In [4]:
HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
}

.app-subtitle a {
    color: #106ba3;
}

.app-subtitle a:hover {
    text-decoration: underline;
}

.app-sidebar p {
    margin-bottom: 1em;
    line-height: 1.7;
}

.app-sidebar a {
    color: #106ba3;
}

.app-sidebar a:hover {
    text-decoration: underline;
}
</style>
""")

In [5]:
class App:

    def __init__(self, df):
        self._df = df
        available_indicators = self._df['Indicator Name'].unique()
        self._x_dropdown = self._create_indicator_dropdown(available_indicators, 0)
        self._y_dropdown = self._create_indicator_dropdown(available_indicators, 1)

        x_scale = LinearScale()
        y_scale = LinearScale()

        self._x_axis = Axis(scale=x_scale, label="X")
        self._y_axis = Axis(scale=y_scale, orientation="vertical", label="Y")

        self._scatter = Scatter(
            x=[], y=[], scales={"x": x_scale, "y": y_scale}
        )

        self._figure = Figure(marks=[self._scatter], axes=[self._x_axis, self._y_axis], layout=dict(width="99%"), animation_duration=1000)

        self._year_slider, year_slider_box = self._create_year_slider(
            min(df['Year']), max(df['Year'])
        )
        _app_container = widgets.VBox([
            widgets.HBox([self._x_dropdown, self._y_dropdown]),
            self._figure,
            year_slider_box
        ], layout=widgets.Layout(align_items='center', flex='3 0 auto'))
        self.container = widgets.VBox([
            widgets.HTML(
                (
                    '<h1>Development indicators. A Voici dashboard, running entirely in your browser!</h1>'
                    '<h2 class="app-subtitle"><a href="https://github.com/pbugnion/voila-gallery/blob/master/country-indicators/index.ipynb">Link to code</a></h2>'
                ),
                layout=widgets.Layout(margin='0 0 5em 0')
            ),
            widgets.HBox([
                _app_container,
                widgets.HTML(EXPLANATION, layout=widgets.Layout(margin='0 0 0 2em'))
            ])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        self._update_app()

    @classmethod
    def from_csv(cls, path):
        df = pd.read_csv(path)
        return cls(df)

    def _create_indicator_dropdown(self, indicators, initial_index):
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index])
        dropdown.observe(self._on_change, names=['value'])
        return dropdown

    def _create_year_slider(self, min_year, max_year):
        year_slider_label = widgets.Label('Year range: ')
        year_slider = widgets.IntRangeSlider(
            min=min_year, max=max_year,
            layout=widgets.Layout(width='500px'),
            continuous_update=False
        )
        year_slider.observe(self._on_change, names=['value'])
        year_slider_box = widgets.HBox([year_slider_label, year_slider])
        return year_slider, year_slider_box

    def _on_change(self, _):
        self._update_app()

    def _update_app(self):
        x_indicator = self._x_dropdown.value
        y_indicator = self._y_dropdown.value
        year_range = self._year_slider.value

        with self._scatter.hold_sync():
            df = self._df[self._df['Year'].between(*year_range)].dropna()
            x = df[df['Indicator Name'] == x_indicator]['Value']
            y = df[df['Indicator Name'] == y_indicator]['Value']

            self._x_axis.label = x_indicator
            self._y_axis.label = y_indicator

            self._scatter.default_opacities = [0.2]

            self._scatter.x = x
            self._scatter.y = y

In [6]:
app = App.from_csv("indicators.csv")

app.container

In [88]:
import bqplot.pyplot as plt
import numpy as np

def transpose(li: list[list]):
    return list(map(list, zip(*li)))

map_colors = {'Heating': '#c22324',
 'Domestic Hot Water': '#ff7f0e',
 'Heating & Domestic Hot Water': '#eb4f18',
 'Cooling': '#1f77b4',
 'Auxiliary': '#9467bd',
 'Lighting': '#17becf',
 'Small Power': '#d3d3d3',
 'IT/Servers': '#8c564b',
 'PV Generation': '#bcbd22',
 'Catering': '#2ca02c',
 'Lifts': '#e377c2',
 'Other': '#000000',
 'Unknown': '#7C7270'}

labels = ['Heating', 'Domestic Hot Water', 'Cooling', 'Auxiliary', 'Lighting', 'Small Power', 'PV Generation']
colors = [map_colors[x] for x in labels]

fig = plt.figure(title="Stacked Bar Chart")
x = [
    "existing building",
    "replace glazing",
    "improve airtightness",
    "replace heat source with an efficient ASHP",
    "update lighting to LED",
    "add PV panels"
]
baseline = [7, 3, 2, 1, 1, 3, 0]
input_y = [
    [7, 3, 2, 1, 1, 3, 0],
    [5, 3, 2, 1, 1, 3, 0],
    [4, 3, 2, 1, 1, 3, 0],
    # [4, 3, 2, 1, 1, 3, 0],
    # [7, 3, 2, 1, 1, 3, 0],
    # [7, 3, 2, 1, 1, 3, 0],
]


y = transpose(input_y)
stacked_bar = plt.bar(x, y, type="stacked", padding=.4, colors=colors, labels=labels, display_legend=True, legend_location="top")
                      

fig

Figure(axes=[Axis(scale=OrdinalScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top'…

In [93]:
import ipywidgets as w

w.SelectMultiple(options=x, layout={"height": "120px"})

SelectMultiple(layout=Layout(height='120px'), options=('existing building', 'replace glazing', 'improve airtig…

In [68]:
stacked_bar.y = transpose(input_y[0:3])

In [66]:
transpose(input_y[0:3])

[[7, 5, 4, 7, 7, 7],
 [3, 3, 3, 3, 3, 3],
 [2, 2, 2, 2, 2, 2],
 [1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1],
 [3, 3, 3, 3, 3, 3],
 [0, 0, 0, 0, 0, 0]]

In [71]:
y

array([0.82076277, 0.69721998, 0.62887945, 0.6341858 , 0.68257821])

In [94]:
import bqplot as bq

# create two vectors x and y to plot a bar chart
x = list("ABCDE")
y = np.random.rand(5)
y = np.random.rand(3, 5)

# 1. Create the scales
xs = bq.OrdinalScale()  # ordinal scale to represent categorical data
ys = bq.LinearScale()

# 2. Create the axes for x and y
xax = bq.Axis(scale=xs, label="X", grid_lines="none")  # no grid lines needed for x
yax = bq.Axis(
    scale=ys, orientation="vertical", label="Y", tick_format=".0%"
)  # note the use of tick_format to format ticks

colors=["orangered", "steelblue", "limegreen"]
# 3. Create a Bars mark by passing in the scales
bar = bq.Bars(x=x, y=y, scales={"x": xs, "y": ys}, padding=0.5, colors=colors, labels=colors, display_legend=True)

# 4. Create a Figure object by assembling marks and axes
bq.Figure(marks=[bar], axes=[xax, yax], title="Bar Chart", legend_location="top")

Figure(axes=[Axis(grid_lines='none', label='X', scale=OrdinalScale()), Axis(label='Y', orientation='vertical',…